In [1]:
import pandas as pd
from itertools import combinations
import numpy as np

<h1>ARM for clusters</h1>

In [8]:
# load and merge all appropriate datasets
df_products   = pd.read_csv("../data_versions/clusters.csv")        # Description  | Cluster
df_labels     = pd.read_csv("../data_versions/cluster_labels.csv")  # Cluster | Label
df_transactions = pd.read_csv('../data_versions/cleaned_1.csv')


df_products_labeled   = df_products.merge(df_labels, on="Cluster", how="outer")  
df_transactions_labeled = df_transactions.merge(df_products_labeled, on="Description", how="outer")

In [9]:
df_transactions_labeled.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cluster,Label
0,536522,72800B,4 PURPLE FLOCK DINNER CANDLES,2,2010-12-01 12:49:00,2.55,15012.0,United Kingdom,22,CANDLE
1,537044,72800B,4 PURPLE FLOCK DINNER CANDLES,12,2010-12-05 10:52:00,2.55,18055.0,United Kingdom,22,CANDLE
2,539595,72800B,4 PURPLE FLOCK DINNER CANDLES,2,2010-12-20 13:43:00,5.06,NaN,United Kingdom,22,CANDLE
3,540247,72800B,4 PURPLE FLOCK DINNER CANDLES,1,2011-01-05 15:56:00,2.55,15464.0,United Kingdom,22,CANDLE
4,542226,72800B,4 PURPLE FLOCK DINNER CANDLES,1,2011-01-26 13:20:00,2.55,17075.0,United Kingdom,22,CANDLE


In [10]:
df_transactions_labeled.to_csv('../data_versions/whole_data.csv') # save finalized dataset

In [11]:
df_transactions_grouped = df_transactions_labeled.groupby("InvoiceNo")['Label'].agg(list).reset_index()
df_transactions_grouped.head()

,InvoiceNo,Label
0,536365,"[HEART, T-LIGHT, MUG, PINK, BOX, T-LIGHT, CANDLE]"
1,536366,"[MUG, MUG]"
2,536367,"[EGG, SET, BOX, BOX, HOME, PINK, HOME, MUG, HO..."
3,536368,"[STAND, BOX, STAND, STAND]"
4,536369,[HOME]


In [12]:
# generate size 2 itemsets
labels = list(df_labels['Label'])
labels[:5]

['MUG', 'GLASS', 'DRAWER', 'CUSHION', 'WRAP']

In [13]:
result = list(combinations(sorted(labels), 2))
result[:5]

[('BAG', 'BOX'),
 ('BAG', 'CANDLE'),
 ('BAG', 'CARD'),
 ('BAG', 'CHRISTMAS'),
 ('BAG', 'CLOCK')]

In [14]:
# construct dictionary that will hold all 2-itemsets and their "count"
rules_count = {}
for r in result:
    rules_count.update({r: 0})

In [15]:
#iterate through the dataset and update rule counts accordingly
for row in range(df_transactions_grouped.shape[0]):
    t = df_transactions_grouped.iloc[row, 1]
    itemsets = combinations(sorted(t), 2)
    for i in itemsets:
        if len(set(i)) > 1: #ignore when 2 items from same cluster are bought
            rules_count[i] += 1

In [16]:
# extract rule frequencies
rules_strength = [rules_count[rule] for rule in rules_count]
rules_strength[:5]

[414945, 125245, 104736, 153349, 33943]

In [17]:
# calculate proportion of transactions that a given rule appears in - this will be the 'strength' factor
rules_strength_array = np.array(rules_strength)
rules_strength_prop = rules_strength_array / df_transactions_grouped.shape[0]

In [18]:
# save rules and their strength to csv
df_rule_strengths = pd.DataFrame(
    {
        'Rule': list(rules_count.keys()),
        'Strength': rules_strength_prop
    }
)

In [ ]:
df_test = pd.read_csv('../data_versions/')

In [ ]:
#df_rule_strengths.to_csv('../data_versions/rules_strengths.csv')

<h1>ARM for products</h1>